In [ ]:
install.packages("crate")

In [2]:
# 1. Load library and set connection details
library(mlflow)

# Set credentials for your local MinIO server
Sys.setenv(
  AWS_ACCESS_KEY_ID      = "root",
  AWS_SECRET_ACCESS_KEY  = "12345678",
  MLFLOW_S3_ENDPOINT_URL = "http://127.0.0.1:9000"
)

# Set the MLflow tracking server URI
mlflow_set_tracking_uri("http://127.0.0.1:5000")

# 2. Run the connection test within a try-catch block
tryCatch({
  
  # Use a specific experiment for this test
  mlflow_set_experiment("Connection_Test")
  
  with(mlflow_start_run(), {
    
    run_id <- mlflow_get_run()$run_uuid
    message(paste("✅ Connected to tracking server. Started run:", run_id))
    
    # Create a simple temporary file to upload
    test_file <- tempfile(fileext = ".txt")
    writeLines("Connection to MinIO successful!", test_file)
    
    # This is the main test: attempting to log the file to MinIO
    mlflow_log_artifact(test_file, artifact_path = "test_data")
    message("✅ Successfully logged artifact to MinIO.")
    
  })
  
  # If we get here, everything worked
  message("\n🎉 SUCCESS! Your setup is configured correctly.")
  
}, error = function(e) {
  
  # If any part of the 'try' block fails, this will run
  message("\n❌ FAILURE! The connection test failed.")
  message("The error was:")
  print(e$message) # Print a cleaner error message
  
})

✅ Connected to tracking server. Started run: 7e07dd86d9a6495ab7ff3fe49f1aef30



Y:\DSW\ML-flow\env\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2025/09/26 14:41:00 INFO mlflow.store.artifact.cli: Logged artifact from local file C:\Users\Admin\AppData\Local\Temp\RtmpwLNhgT\file5d303d945340.txt to artifact_path=test_data


Root URI: mlflow-artifacts:/2/7e07dd86d9a6495ab7ff3fe49f1aef30/artifacts

✅ Successfully logged artifact to MinIO.


🎉 SUCCESS! Your setup is configured correctly.



In [4]:
library(mlflow)
library(carrier)
Sys.setenv(
  AWS_ACCESS_KEY_ID = "root",
  AWS_SECRET_ACCESS_KEY = "12345678",
  MLFLOW_S3_ENDPOINT_URL = "http://127.0.0.1:9000"
)
# Load iris dataset
data("iris")
mlflow_set_tracking_uri("http://127.0.0.1:5000")
mlflow_set_experiment("MLflow_Quickstart_R")
set.seed(123)
# Learn simple linear regression model
model <- lm(Sepal.Width~Sepal.Length, data = iris)
Sys.setenv(
  AWS_ACCESS_KEY_ID = "root",
  AWS_SECRET_ACCESS_KEY = "12345678",
  MLFLOW_S3_ENDPOINT_URL = "http://127.0.0.1:9000"
)

# Define a crate model
# call package functions with an explicit :: namespace.
crate_model <- crate(
  function(new_obs)  stats::predict(model, data.frame("Sepal.Length" = new_obs)),
  model = model
)

# log the model
model_path <- mlflow_log_model(model = crate_model, artifact_path = "iris_prediction")

# load the logged model and make a prediction
model_uri <- paste0(mlflow_get_run()$artifact_uri, "/iris_prediction")
mlflow_model <- mlflow_load_model(model_uri = model_uri,
                                  flavor = NULL,
                                  client = mlflow_client())

prediction <- mlflow_predict(model = mlflow_model, data = 5)
print(prediction)



[1] "1"

Y:\DSW\ML-flow\env\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2025/09/26 14:43:17 INFO mlflow.store.artifact.cli: Logged artifact from local dir C:/Users/Admin/AppData/Local/Temp/RtmpwLNhgT/iris_prediction to artifact_path=iris_prediction


Root URI: mlflow-artifacts:/1/d3a9317e96bd4c7faa1c3f94b1dc0b30/artifacts



       1 
3.109523 


In [7]:
library(mlflow)
library(carrier)
library(randomForest)
library(MASS)  # for Boston dataset

# Point MLflow to your tracking server
mlflow_set_tracking_uri("http://127.0.0.1:5000")
mlflow_set_experiment("MLflow_Quickstart_R")

set.seed(123)

# ---- Data ----
data(Boston)
train_idx <- sample(1:nrow(Boston), 0.8 * nrow(Boston))
train <- Boston[train_idx, ]
test  <- Boston[-train_idx, ]

# ---- Train RandomForest ----
model_rf <- randomForest(medv ~ ., data = train)

# ---- Evaluate ----
pred_rf <- predict(model_rf, newdata = test)
mse_rf <- mean((pred_rf - test$medv)^2)
cat("MSE:", mse_rf, "\n")

# ---- Log with MLflow ----
with(mlflow_start_run(), {
  mlflow_log_param("model_type", "random_forest")
  mlflow_log_metric("mse", mse_rf)

  # ✅ Wrap model in crate (explicit namespace for predict)
rf_crate <- carrier::crate(
  function(df) {
    randomForest:::predict.randomForest(model_rf, df)
  },
  model_rf = model_rf
)

  # Log crate model
  mlflow_log_model(rf_crate, artifact_path = "rf_model")

  # ---- Load back model from MLflow ----
  model_uri <- paste0(mlflow_get_run()$artifact_uri, "/rf_model")
  mlflow_model <- mlflow_load_model(model_uri = model_uri,
                                    flavor = NULL,
                                    client = mlflow_client())

  # ---- Test reload + prediction ----
  sample_input <- test[1:5, -which(names(test) == "medv")]  # features only
  prediction <- mlflow_predict(model = mlflow_model, data = sample_input)
  print(prediction)
})


mlflow_end_run()
print("done")

[1] "1"

MSE: 11.38223 
Y:\DSW\ML-flow\env\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2025/09/26 14:59:33 INFO mlflow.store.artifact.cli: Logged artifact from local dir C:/Users/Admin/AppData/Local/Temp/RtmpwLNhgT/rf_model to artifact_path=rf_model


Root URI: mlflow-artifacts:/1/d2cbc5aedb1b43d79da1f989b437028c/artifacts



       1       15       17       19       28 
28.48333 19.02469 21.12301 18.40968 15.17559 


ERROR: Error: There is no active run to end.


In [6]:
mlflow_end_run()

run_uuid                         experiment_id run_name          user_id
1 d3a9317e96bd4c7faa1c3f94b1dc0b30 1             abundant-newt-772 Admin  
2 d3a9317e96bd4c7faa1c3f94b1dc0b30 1             abundant-newt-772 Admin  
3 d3a9317e96bd4c7faa1c3f94b1dc0b30 1             abundant-newt-772 Admin  
4 d3a9317e96bd4c7faa1c3f94b1dc0b30 1             abundant-newt-772 Admin  
5 d3a9317e96bd4c7faa1c3f94b1dc0b30 1             abundant-newt-772 Admin  
  status   start_time          end_time           
1 FINISHED 2025-09-26 14:43:07 2025-09-26 14:59:18
2 FINISHED 2025-09-26 14:43:07 2025-09-26 14:59:18
3 FINISHED 2025-09-26 14:43:07 2025-09-26 14:59:18
4 FINISHED 2025-09-26 14:43:07 2025-09-26 14:59:18
5 FINISHED 2025-09-26 14:43:07 2025-09-26 14:59:18
  artifact_uri                                                  
1 mlflow-artifacts:/1/d3a9317e96bd4c7faa1c3f94b1dc0b30/artifacts
2 mlflow-artifacts:/1/d3a9317e96bd4c7faa1c3f94b1dc0b30/artifacts
3 mlflow-artifacts:/1/d3a9317e96bd4c7faa1c3f94b1dc0b30/artifacts
4 mlflow-artifacts:/1/d3a9317e96bd4c7faa1c3f94b1dc0b30/artifacts
5 mlflow-artifacts:/1/d3a9317e96bd4c7faa1c3f94b1dc0b30/artifacts
  lifecycle_stage run_id                           metrics params
1 active          d3a9317e96bd4c7faa1c3f94b1dc0b30 NA      NA    
2 active          d3a9317e96bd4c7faa1c3f94b1dc0b30 NA      NA    
3 active          d3a9317e96bd4c7faa1c3f94b1dc0b30 NA      NA    
4 active          d3a9317e96bd4c7faa1c3f94b1dc0b30 NA      NA    
5 active          d3a9317e96bd4c7faa1c3f94b1dc0b30 NA      NA    
  tags                    
1 mlflow.user             
2 mlflow.source.name      
3 mlflow.source.type      
4 mlflow.runName          
5 mlflow.log-model.history
  <NA>                                                                                                                                                                                                                        
1 Admin                                                                                                                                                                                                                       
2 <console>                                                                                                                                                                                                                   
3 LOCAL                                                                                                                                                                                                                       
4 abundant-newt-772                                                                                                                                                                                                           
5 [{"run_id": "d3a9317e96bd4c7faa1c3f94b1dc0b30", "artifact_path": "iris_prediction", "utc_time_created": "2025-09-26 09:13:07.320911", "model_uuid": null, "flavors": {"crate": {"version": "0.1.0", "model": "crate.bin"}}}]